In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime
import pytz
import openpyxl


In [2]:
# download the rk csvs as dataframes
rk_price_down = pd.read_csv("markets-data/rk/new_rk_price_down.csv")
rk_price_up = pd.read_csv("markets-data/rk/new_rk_price_up.csv")
rk_volume_up = pd.read_csv("markets-data/rk/new_rk_vol_up.csv")
rk_volume_down = pd.read_csv("markets-data/rk/new_rk_vol_down.csv")

rk_price_down.drop(columns = ["currency"], inplace=True)
rk_price_up.drop(columns = ["currency"], inplace=True)
"""rk_volume_up.drop(columns = ["bi_created"], inplace=True)
rk_volume_down.drop(columns = ["bi_created"], inplace=True)"""


'rk_volume_up.drop(columns = ["bi_created"], inplace=True)\nrk_volume_down.drop(columns = ["bi_created"], inplace=True)'

In [4]:
print(rk_volume_down["start_time"].iloc[0])
print(type(rk_volume_down["start_time"].iloc[0]))
#print(rk_volume_down["start_time"].iloc[0].tzinfo)
print(len(rk_volume_down))

2022-01-01T00:00:00.000Z
<class 'str'>
77602


In [13]:
rk_dfs_dict = {"price_down" : rk_price_down,"price_up" : rk_price_up,"volume_up" : rk_volume_up,"volume_down" : rk_volume_down}

In [6]:
for name in rk_dfs_dict.keys():
    print(name, " : ", len(rk_dfs_dict[name]["value"]))
    for index, val in enumerate(rk_dfs_dict[name]["value"].isna()):
        #print(val)
        if val == True:
            print(index, " : ", name)

price_down  :  78820
price_up  :  78814
volume_up  :  77602
volume_down  :  77602


In [9]:
def preprocess_dfs_dict(df_dict : dict, area : str, start_month : int, start_year : int, start_day : int, start_hour : int, end_hour : int, end_month : int, end_year : int, end_day : int):
    start_datetime = pd.Timestamp(year = start_year, month= start_month, day=start_day, hour= start_hour, tz = "Europe/Oslo") #Europe/Oslo    
    end_datetime = pd.Timestamp(year = end_year, month= end_month, day=end_day, hour= end_hour, tz = "Europe/Oslo")
    updated_df_dict = {}
    for name in df_dict.keys():
        df = df_dict[name]
        #df.drop(columns = ["end_time", "bi_created", "start_year" ], inplace = True)
        df["start_time"] = pd.to_datetime(df["start_time"], format="%Y-%m-%d %H:%M:%S")
        #df["start_time"] = df["start_time"].dt.tz_localize("UTC")
        df["start_time"] = df["start_time"].dt.tz_convert("Europe/Oslo")
        df.sort_values(by = ["start_time", "delivery_area"], inplace = True)
        df.rename(columns = {"start_time" : "Time(Local)", "value" : name}, inplace = True)
        filtered_df = df[(df["Time(Local)"] >= start_datetime) & (df["Time(Local)"] < end_datetime) & (df["delivery_area"] == area)]
        filtered_df.sort_values(by = ["Time(Local)"], inplace = True)
        updated_df_dict[name] = filtered_df
        
    return updated_df_dict

In [14]:
new_dict = preprocess_dfs_dict(rk_dfs_dict, "NO5",start_year= 2023, end_year = 2023, start_month = 6, end_month = 6, start_day = 25, end_day = 27, start_hour = 0, end_hour = 0)

/var/folders/zq/tp7687p50wq08mhvgw4wlv780000gn/T/ipykernel_85490/3036011239.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = ["Time(Local)"], inplace = True)
/var/folders/zq/tp7687p50wq08mhvgw4wlv780000gn/T/ipykernel_85490/3036011239.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = ["Time(Local)"], inplace = True)
/var/folders/zq/tp7687p50wq08mhvgw4wlv780000gn/T/ipykernel_85490/3036011239.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/panda

In [16]:
new_dict["price_down"]

,Time(Local),price_down,delivery_area
64794,2023-06-25 00:00:00+02:00,49.47,NO5
64799,2023-06-25 01:00:00+02:00,41.79,NO5
64804,2023-06-25 02:00:00+02:00,39.50,NO5
64809,2023-06-25 03:00:00+02:00,40.50,NO5
64814,2023-06-25 04:00:00+02:00,38.75,NO5
64819,2023-06-25 05:00:00+02:00,40.50,NO5
64824,2023-06-25 06:00:00+02:00,49.37,NO5
64829,2023-06-25 07:00:00+02:00,48.00,NO5
64834,2023-06-25 08:00:00+02:00,48.00,NO5
64839,2023-06-25 09:00:00+02:00,25.00,NO5


In [12]:
def preprocess_dfs(df_dict):
    copy_dict = df_dict.copy()
    for name in df_dict.keys():
        df = copy_dict[name]
        #df.drop(columns = ["end_time", "bi_created", "start_year" ], inplace = True)
        df["start_time"] = pd.to_datetime(df["start_time"], format="%Y-%m-%d %H:%M:%S")
        #df["start_time"] = df["start_time"].dt.tz_localize("UTC")
        df["start_time"] = df["start_time"].dt.tz_convert("Europe/Oslo")
        df.sort_values(by = ["start_time", "delivery_area"], inplace = True)
        df.rename(columns = {"start_time" : "Time(Local)", "value" : name}, inplace = True)
    return copy_dict

In [8]:
updated_df_dict = preprocess_dfs(rk_dfs_dict)

In [9]:
print(updated_df_dict["price_down"]["Time(Local)"].iloc[0])
print(type(updated_df_dict["price_down"]["Time(Local)"].iloc[0]))
print(updated_df_dict["price_down"]["Time(Local)"].iloc[0].tzinfo)

2022-01-01 01:00:00+01:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Europe/Oslo


In [12]:
def find_missing_hours(df : pd.DataFrame, area : str):
    # Define the year for which you want to find missing hours
    # Generate a reference array with all hours in a year
    start_date = np.datetime64("2022-01-01T00:00:00")
    end_date = np.datetime64("2023-06-30T23:00:00")
    reference_dates = np.arange(start_date, end_date, dtype='datetime64[h]')
    #print(len(reference_dates))
    #print(reference_dates)

    # Convert your actual datetime_array to a set of timestamps
    actual_dates = df["Time(Local)"].loc[df["delivery_area"] == area].values

    # Find missing hours by comparing the reference and actual arrays
    missing_hours = [date for date in reference_dates if date not in actual_dates]

    """print("Missing hours: ", len(missing_hours))
    for missing_hour in missing_hours:
        print(missing_hour)"""
    return missing_hours

In [13]:
find_missing_hours(rk_volume_down, "NO3")

[numpy.datetime64('2022-11-17T00','h')]

In [14]:
def get_missing_values(df_dict, area):
    missing_df = {}
    for key in df_dict.keys():
        missing_df[key] = find_missing_hours(df_dict[key], area)
    return missing_df
  

In [15]:
missing_dict_NO1 = get_missing_values(updated_df_dict, "NO1")
missing_dict_NO2 = get_missing_values(updated_df_dict, "NO2")
missing_dict_NO3 = get_missing_values(updated_df_dict, "NO3")
missing_dict_NO4 = get_missing_values(updated_df_dict, "NO4")
missing_dict_NO5 = get_missing_values(updated_df_dict, "NO5")

In [16]:
print("Missing values for the different markets in NO1 : ", missing_dict_NO1)
print("\n")
print("Missing values for the different markets in NO2 : ", missing_dict_NO2)
print("\n")
print("Missing values for the different markets in NO3 : ", missing_dict_NO3)
print("\n")
print("Missing values for the different markets in NO4 : ", missing_dict_NO4)
print("\n")
print("Missing values for the different markets in NO5 : ", missing_dict_NO5)


Missing values for the different markets in NO1 :  {'price_down': [], 'price_up': [], 'volume_up': [numpy.datetime64('2022-06-23T05','h'), numpy.datetime64('2022-06-23T06','h'), numpy.datetime64('2022-11-16T23','h'), numpy.datetime64('2022-11-17T00','h')], 'volume_down': [numpy.datetime64('2022-06-23T05','h'), numpy.datetime64('2022-06-23T06','h'), numpy.datetime64('2022-11-16T23','h'), numpy.datetime64('2022-11-17T00','h')]}


Missing values for the different markets in NO2 :  {'price_down': [], 'price_up': [], 'volume_up': [numpy.datetime64('2022-06-23T05','h'), numpy.datetime64('2022-06-23T06','h'), numpy.datetime64('2022-11-17T00','h')], 'volume_down': [numpy.datetime64('2022-06-23T05','h'), numpy.datetime64('2022-06-23T06','h'), numpy.datetime64('2022-11-17T00','h')]}


Missing values for the different markets in NO3 :  {'price_down': [], 'price_up': [], 'volume_up': [numpy.datetime64('2022-11-17T00','h')], 'volume_down': [numpy.datetime64('2022-11-17T00','h')]}


Missing values f

In [53]:
def get_datebased_rk_df(rk_dict: dict, area : str, start_month : int, start_year : int, start_day : int, start_hour : int, end_hour : int, end_month : int, end_year : int, end_day : int):
    
    start_datetime = pd.Timestamp(year = start_year, month= start_month, day=start_day, hour= start_hour, tz = "Europe/Oslo") #Europe/Oslo
    print(start_datetime)
    
    end_datetime = pd.Timestamp(year = end_year, month= end_month, day=end_day, hour= end_hour, tz = "Europe/Oslo")
    print(end_datetime)
    new_df_list = []
    for name in rk_dict.keys():
        df = rk_dict[name]
        #df.sort_values(by = ["Time(Local)"], inplace = True)
        filtered_df = df[(df["Time(Local)"] >= start_datetime) & (df["Time(Local)"] < end_datetime) & (df["delivery_area"] == area)]
        filtered_df.sort_values(by = ["Time(Local)"], inplace = True)
        new_df_list.append(filtered_df)
        
    
    return new_df_list

In [54]:
rk_list = get_datebased_rk_df(updated_df_dict, "NO5", start_year= 2023, start_month= 6, start_day = 26, start_hour = 0, end_hour = 0, end_year = 2023, end_month = 6, end_day = 27)

2023-06-26 00:00:00+02:00
2023-06-27 00:00:00+02:00


/var/folders/zq/tp7687p50wq08mhvgw4wlv780000gn/T/ipykernel_73464/4212979475.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = ["Time(Local)"], inplace = True)


In [55]:
rk_list[3]

,Time(Local),volume_down,delivery_area
64901,2023-06-26 00:00:00+02:00,0.0,NO5
64906,2023-06-26 01:00:00+02:00,0.0,NO5
64911,2023-06-26 02:00:00+02:00,0.0,NO5
64916,2023-06-26 03:00:00+02:00,0.0,NO5
64921,2023-06-26 04:00:00+02:00,0.0,NO5
64926,2023-06-26 05:00:00+02:00,0.0,NO5
64931,2023-06-26 06:00:00+02:00,0.0,NO5
64936,2023-06-26 07:00:00+02:00,0.0,NO5
64941,2023-06-26 08:00:00+02:00,0.0,NO5
64946,2023-06-26 09:00:00+02:00,0.0,NO5
